In [ ]:
# Sources Multi Color:
#https://www.dca.ufrn.br/~adelardo/artigos/SAC08.pdf

#Sources Robust Estimation:
#https://github.com/jakecerwin/LucasKanade/blob/d8631ac88b8836b80e596ec298f6a5516fafd1c1/LucasKanade_Pyramid.py
import cv2 as cv